In [2]:
import pandas as pd

names = pd.read_csv("/Users/kritinshanmugam/Desktop/names.csv", header=None)
names_list = names.iloc[:, 0].tolist()

X = pd.read_csv("/Users/kritinshanmugam/Desktop/genes.csv", header=None, names=names_list)
y = pd.read_csv("/Users/kritinshanmugam/Desktop/labels.csv", header=None, names = ['Label'])

In [3]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

label_encoder = LabelEncoder()

# Handle missing values
X = X.dropna()
y = y.dropna()

# Standardize features
X = (X - X.mean()) / X.std()

# Create copy of original (untransformed) y data to compare later with predictions
y_orig = y.copy()

print(X.head())

   0610007P14Rik  0610009B22Rik  0610009L18Rik  0610009O20Rik  0610010F05Rik  \
0       6.357779      -0.129192      -0.133012      -0.339486      -0.441585   
1      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
2      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
3      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   
4      -0.177072      -0.129192      -0.133012      -0.339486      -0.441585   

   0610012D04Rik  0610012G03Rik  0610025J13Rik  0610030E20Rik  0610031O16Rik  \
0      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
1      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
2      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
3      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   
4      -0.048584      -0.218664      -0.071389      -0.290955      -0.011646   

   ...    mt-Co2    mt-Co3   mt-Cytb  

In [5]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

'''
# Create list to store incorrect predictions
xg_incorrect = []

for row in range(len(xg_preds)):
    if xg_preds.iloc[row, 0] != y_test.iloc[row, 0]:
        xg_incorrect.append(row)
        
print(len(xg_incorrect))
'''

# List to store each XGBoost model
models = []


# Create 100 models, train on 90% of dataset, find model with lowest score
for i in range(2):
    
    print(i)
    
    # Create temporary XGBoost instance
    temp_model = XGBClassifier()
    
    # Split into 90% training, 10% test
    t_x_train, t_x_test, t_y_train, t_y_test = train_test_split(X, y, test_size=0.1)
    
    t_y_test = pd.DataFrame(t_y_test)
    actual = t_y_test.values
    
    # Train the model
    temp_model.fit(t_x_train, t_y_train)
    
    # Predict based on x_test
    temp_pred = temp_model.predict(t_x_test)
    temp_pred = pd.DataFrame(temp_pred)
    
    # Get F1 score
    temp_f1 = f1_score(actual, temp_pred.values)
    
    # Add the model to the list for storage
    models.append((temp_model, temp_f1))
    

sorted_models = sorted(models, key=lambda x: x[1], reverse=True)


0
1
[(XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 0.8707607699358386), (XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
             

In [10]:
'''
optimal_hundred = sorted_models[:99]

optimal_predictions = []
for model in optimal_hundred:
    pred = model.predict(X)
    optimal_predictions.append(pred)

pred_array = np.array(optimal_predictions)
mispredicted_count = np.sum(pred_array != np.array(y), axis=0)
bad_points = np.where(mispredicted_count > 50)[0]
'''

temp = [[1,2,3], [1,3,1], [1,6,1]]
temp_real = [1,1,1]
temp_ar = np.array(temp)
mis_temp = np.sum(temp_ar != np.array(y), axis = 0)
bad_temp = np.where(mis_temp > 50)[0]
print(bad_temp)

[]


/var/folders/fy/m0zjz52x4kb7hwy0z4jxwk5r0000gn/T/ipykernel_35241/1071011410.py:17: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  mis_temp = np.sum(temp_ar != np.array(y), axis = 0)


In [ ]:
'''
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Get the actual values and predicted values in array-like format
actual = y_test.values

predicted = xg_preds.values

# Generate a confusion matrix
confusion = confusion_matrix(actual, predicted, labels=xg_classifier.classes_) 

# Generate display for confusion matrix
display = ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=xg_classifier.classes_)

display.plot()
'''

In [ ]:
import shap

explainer = shap.TreeExplainer(classifier, x_test)
shap_values = explainer(x_test)

for row in incorrect_list:
    shap.plots.waterfall(shap_values[row])